<a href="https://colab.research.google.com/github/alejovillores/tp2_spark_villores_alejo/blob/main/ejercicio_23.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Ejercicio 23
Cantidad total de contenidos por tipo de locación que pertenecen a la tierra

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq
#!apt install default-jre
#!apt install default-jdk
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 281.3 MB 36 kB/s 
     |████████████████████████████████| 198 kB 48.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=8cdfafcf6baeceb48306b79eda10275caa00e7f42ec25fcc537e752e68784eeb
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 https://developer.download

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [3]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [5]:
sqlContext = SQLContext(sc)
df = sqlContext.read.parquet('/content/drive/MyDrive/2C2021/Datos/Trabajo Practico 2/geo_tags.csv', header=True, inferSchema=True)
rdd_geo_tags = df.rdd

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [6]:
rdd_geo_tags.take(10)

[Row(gt_id=158041, gt_page_id=4328020, gt_globe='earth', gt_primary=0, gt_lat=41.50749969, gt_lon=-5.81750011, gt_dim=1000, gt_type=None, gt_name=None, gt_country=None, gt_region=None),
 Row(gt_id=5844377, gt_page_id=4670424, gt_globe='earth', gt_primary=0, gt_lat=13.73799992, gt_lon=-89.29199982, gt_dim=10000, gt_type='city', gt_name=None, gt_country=None, gt_region=None),
 Row(gt_id=5844381, gt_page_id=4670424, gt_globe='earth', gt_primary=0, gt_lat=13.70199966, gt_lon=-89.20800018, gt_dim=10000, gt_type='city', gt_name=None, gt_country=None, gt_region=None),
 Row(gt_id=5844383, gt_page_id=4670424, gt_globe='earth', gt_primary=0, gt_lat=13.39999962, gt_lon=-89.75, gt_dim=10000, gt_type='city', gt_name=None, gt_country=None, gt_region=None),
 Row(gt_id=5844388, gt_page_id=4670424, gt_globe='earth', gt_primary=0, gt_lat=12.85000038, gt_lon=-88.0, gt_dim=10000, gt_type='city', gt_name=None, gt_country=None, gt_region=None),
 Row(gt_id=5844393, gt_page_id=4670424, gt_globe='earth', gt_pr

In [9]:
rdd_geo_tags_contents_cleaned = rdd_geo_tags.filter(lambda x: x.gt_type != None)

Filtro los contenidos hechos en la Tierra y cuento la cantidad segun del tipo de locación.

In [10]:
earth_contents = rdd_geo_tags_contents_cleaned.filter(lambda x: x.gt_globe == 'earth').map(lambda x: (x.gt_type, 1))\
.reduceByKey(lambda x,y: x+y)

earth_contents.take(10)

[('landmark', 97436),
 ('airport', 4638),
 ('adm2nd', 334),
 ('forrest', 9562),
 ('bridge', 1136),
 ('railwaystation', 10681),
 ('glacier', 319),
 ('edu', 3151),
 ('event', 5211),
 ('adm1st', 315)]